In [21]:
import json, os
root_dir = "../data"
MRBenchv1 = os.path.join(root_dir, "MRBench/MRBench_V1.json")
MRBenchv2 = os.path.join(root_dir, "MRBench/MRBench_V2.json")

MRBenchv1_data = json.load(open(MRBenchv1))
MRBenchv2_data = json.load(open(MRBenchv2))
print(len(MRBenchv1_data), len(MRBenchv2_data))
MRBenchv1_data[0]

192 200


{'conversation_id': '930-b01cb51d-748d-460c-841a-08e4d5cd5cc7',
 'conversation_history': 'Tutor: Hi, could you please provide a step-by-step solution for the question below? The question is: Elliott is trying to walk 10,000 steps a day. He finished half of his steps on his walks to and from school and did another 1,000 steps going for a short walk with his friend. He also went for a short jog around the block and realized that after he had finished his jog, he only had 2,000 steps left to take. How many steps did Elliott take during his jog?\xa0\n\xa0Student: Elliott finished half of his steps on his walks to and from school, so he took 10,000/2 = 5000 steps during these walks.\nAdding the 1,000 steps he took with his friend, he has taken 5000+1000 = 6000 steps.\nSubtracting 6000 from his goal of 10,000, he has 10,000-6000 = 4000 steps left to take.\nTherefore, he took 4000 steps during his jog.\xa0\n\xa04000\xa0\n\xa0Tutor: can you tell me how you got to your answer?\xa0\n\xa0Student:

MRBench_V1: The original dataset with 192 dialogues as deatiled in the paper.

MRBench_V2: An updated version with additional 8 dialogues, bringing the total to 200 examples.

In [22]:
def map_annotation_label(key, label):
    Tutor_tone_mapping = {
        "encouraging": 1,
        "neutral": 2,
        "offensive": 3
    }
    Other_rule_mapping = {
        "yes": 1,
        "to some extent": 2,
        "no": 3
    }
    map_dict = Tutor_tone_mapping if key == "Tutor_Tone" else Other_rule_mapping
    for key, value in map_dict.items():
        # if key in label.lower():
        #     return value
        if label.lower().startswith(key):
            return value
    print(label)
    return None
    
    
MRBenchv1_data_mapped = []
for data in MRBenchv1_data:
    for key, value in data['anno_llm_responses'].items():
        annotation = value['annotation']
        new_annotation = {}
        for k, v in annotation.items():
            new_annotation[k] = map_annotation_label(k,v)
        value['annotation_point'] = new_annotation
    MRBenchv1_data_mapped.append(data)
print(MRBenchv1_data_mapped[0])
    
     

{'conversation_id': '930-b01cb51d-748d-460c-841a-08e4d5cd5cc7', 'conversation_history': 'Tutor: Hi, could you please provide a step-by-step solution for the question below? The question is: Elliott is trying to walk 10,000 steps a day. He finished half of his steps on his walks to and from school and did another 1,000 steps going for a short walk with his friend. He also went for a short jog around the block and realized that after he had finished his jog, he only had 2,000 steps left to take. How many steps did Elliott take during his jog?\xa0\n\xa0Student: Elliott finished half of his steps on his walks to and from school, so he took 10,000/2 = 5000 steps during these walks.\nAdding the 1,000 steps he took with his friend, he has taken 5000+1000 = 6000 steps.\nSubtracting 6000 from his goal of 10,000, he has 10,000-6000 = 4000 steps left to take.\nTherefore, he took 4000 steps during his jog.\xa0\n\xa04000\xa0\n\xa0Tutor: can you tell me how you got to your answer?\xa0\n\xa0Student: 

In [23]:

desiderata = {
    "Mistake_Identification": 1,  # Yes
    "Mistake_Location": 1,        # Yes
    "Revealing_of_the_Answer": 3,        # No
    "Providing_Guidance": 1,      # Yes
    "Actionability": 1,           # Yes
    "Coherence": 1,                # Yes
    "Tutor_Tone": 1,              # Encouraging
    "humanlikeness": 1,               # Yes
}

# new_annotation
def evaluate_ordinay_desiderata(data):
    from collections import defaultdict
    evaluation_result = defaultdict(dict)
    for data in MRBenchv1_data:
        for model, value in data['anno_llm_responses'].items():
            annotation_point = value['annotation_point']
            for k, v in annotation_point.items(): 
                if v is None:
                    continue
                if v == desiderata[k]:
                    if k not in evaluation_result[model]:
                        evaluation_result[model][k] = [0,0]
                    evaluation_result[model][k][0] += 1
                else:
                    if k not in evaluation_result[model] and v is not None:
                        evaluation_result[model][k] = [0,0]
                    evaluation_result[model][k][1] += 1
    return evaluation_result

evaluation_result = evaluate_ordinay_desiderata(MRBenchv1_data)


In [24]:
def print_evaluation_result(evaluation_result):
    import pandas as pd
    pd_result = []
    columns = []
    for model, value in evaluation_result.items():
        model_result = []
        value = sorted(value.items(), key=lambda x: x[0])
        columns = [k for k, v in value]
        for k, v in value:
            model_result.append((v[0]/(v[0]+v[1] )* 100.0))
        pd_result.append([model] + model_result)  
    columns = ['Tutor'] + columns
    pd_result = pd.DataFrame(pd_result, columns=columns)
    print(pd_result.columns)
    columns_mapping = {
        'Mistake_Identification': 'Mistake_Identification',
        'Mistake_Location': 'Mistake_Location',
        'Revealing_of_the_Answer': 'Revealing_of_the_Answer',
        'Providing_Guidance': 'Providing_Guidance',
        'Actionability': 'Actionability',
        'Coherence': 'Coherence',
        'Tutor_Tone': 'Tutor_Tone',
        'humanlikeness': 'Human-likeness'
    }
    pd_result.rename(columns=columns_mapping, inplace=True)
    pd_result = pd_result[['Tutor', 'Mistake_Identification', 'Mistake_Location', 'Revealing_of_the_Answer', 'Providing_Guidance', 'Actionability', 'Coherence', 'Tutor_Tone', 'Human-likeness']].round(2)
    return pd_result

pd_result=print_evaluation_result(evaluation_result)


Index(['Tutor', 'Actionability', 'Coherence', 'Mistake_Identification',
       'Mistake_Location', 'Providing_Guidance', 'Revealing_of_the_Answer',
       'Tutor_Tone', 'humanlikeness'],
      dtype='object')


In [25]:
import pandas as pd
ss = pd.read_csv('../paper/paper_result.csv',sep='\t')
ss['Tutor'] = ss['Tutor'].apply(lambda x: x.replace("*","")+"_paper")
concat_result = pd.concat([ss, pd_result], axis=0)
concat_result = concat_result.sort_values(by='Tutor')
concat_result.head()

,Tutor,Mistake_Identification,Mistake_Location,Revealing_of_the_Answer,Providing_Guidance,Actionability,Coherence,Tutor_Tone,Human-likeness
5,Expert,81.25,68.75,97.92,72.92,81.77,84.90,17.19,94.79
1,Expert_paper,76.04,63.02,90.62,67.19,76.04,79.17,92.19,87.50
7,GPT-4_paper,94.27,84.38,53.12,76.04,46.35,90.17,37.50,89.62
6,GPT4,94.27,85.42,54.69,77.08,46.88,92.71,36.98,93.23
0,Gemini,87.50,62.50,92.71,58.85,61.98,82.29,39.58,95.31
